In [1]:
import tensorflow
import stellargraph
import numpy
import matplotlib
import os.path
import pandas
import datetime
import re

Please notice, to open jupyter in custom directory use:
>jupyter notebook --notebook-dir=[your_directory]

Expecting file hierarchy:
    <ul>root/
        <li>Antarctica_research_2020/
            <ul>.git/</ul>
            <ul>.gitignore</ul>
            <ul>readme.md</ul>
            <ul>notebook.ipynb</ul></li>
        <li>data/
            <ul>Sesja 1.xlsx</ul>
            <ul>.</ul>
            <ul>.</ul>
            <ul>Sesja 26.xlsx</ul></li>
    </ul>

In [2]:
data_path = "../data/"
file_list = [name for name in os.listdir(data_path)]
print(file_list)

['Sesja 1.xlsx', 'Sesja 10.xlsx', 'Sesja 11.xlsx', 'Sesja 12.xlsx', 'Sesja 13.xlsx', 'Sesja 14.xlsx', 'Sesja 15.xlsx', 'Sesja 16.xlsx', 'Sesja 17.xlsx', 'Sesja 18.xlsx', 'Sesja 19.xlsx', 'Sesja 2.xlsx', 'Sesja 20.xlsx', 'Sesja 21.xlsx', 'Sesja 22.xlsx', 'Sesja 23.xlsx', 'Sesja 24.xlsx', 'Sesja 25.xlsx', 'Sesja 26.xlsx', 'Sesja 3.xlsx', 'Sesja 4.xlsx', 'Sesja 5.xlsx', 'Sesja 6.xlsx', 'Sesja 7.xlsx', 'Sesja 8.xlsx', 'Sesja 9.xlsx']


In [3]:
# Sessions order is important, let me get sorted file info
sorted_filenames = dict()
for name in file_list:
    list_of_digits = list(map(int, filter(str.isdigit, name)))
    sorted_filenames[name] = list_of_digits[0] if len(list_of_digits) == 1 else list_of_digits[1] + list_of_digits[0]*10
sorted_filenames = sorted(sorted_filenames.items(), key=lambda el: el[1])
sorted_filenames = {val: key for key, val in sorted_filenames}
print(sorted_filenames)

{1: 'Sesja 1.xlsx', 2: 'Sesja 2.xlsx', 3: 'Sesja 3.xlsx', 4: 'Sesja 4.xlsx', 5: 'Sesja 5.xlsx', 6: 'Sesja 6.xlsx', 7: 'Sesja 7.xlsx', 8: 'Sesja 8.xlsx', 9: 'Sesja 9.xlsx', 10: 'Sesja 10.xlsx', 11: 'Sesja 11.xlsx', 12: 'Sesja 12.xlsx', 13: 'Sesja 13.xlsx', 14: 'Sesja 14.xlsx', 15: 'Sesja 15.xlsx', 16: 'Sesja 16.xlsx', 17: 'Sesja 17.xlsx', 18: 'Sesja 18.xlsx', 19: 'Sesja 19.xlsx', 20: 'Sesja 20.xlsx', 21: 'Sesja 21.xlsx', 22: 'Sesja 22.xlsx', 23: 'Sesja 23.xlsx', 24: 'Sesja 24.xlsx', 25: 'Sesja 25.xlsx', 26: 'Sesja 26.xlsx'}


In [20]:
# Loading data
data_dict = dict()
for key, val in sorted_filenames.items():
    data_dict[key] = pandas.read_excel(data_path + val, header=None, dtype=object)

In [21]:
rand_int = int(numpy.random.rand() * 26)
data_dict[rand_int].head(3)
# An example:

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,NaN,Sesja 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,N = 19,NaN,NaN,1979-01-14 00:00:00,NaN,NaN,NaN,ZIM2,NaN,NaN
1,Nr O.B.,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
2,1,2023,1823,1527,1520,1516,1717,1816,2118,2115,...,2328,1932,2130,1925,2223,1423,1729,1225,2127,NaN


The target format for data is a stellargraph graph for each session (session -> data recording occuring every two week). But before that, let me read the data into intermediate objects:

In [22]:
class Session(object):
    def __init__(self, raw_session_data):
        self.date = self.find_date(raw_session_data)
        self.relations = dict() # dictionary of lists of tuples - column_number : [(x1, y1), (x2, y2), ...]
    
    def get_colums(self):
        pass
    
    def find_date(self, raw_session_data) -> datetime.date:
        for key, col in raw_session_data.iteritems():
            for val in col:
                regex = re.search(r"\d{1,2}.\d{1,2}.\d{4}|\d{4}.\d{1,2}.\d{1,2}", str(val))
                if regex:
                    return regex.group()

In [23]:
class Person(object):
    def __init__(self, id, date, relations: list):
        pass

In [24]:
test_session = Session(data_dict[1])
print(test_session.date)

nan
Nr O.B.
1
2
3
4
5
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
nan
Nr 10 = lider formalny
nan
1
2420
2215
2021
2713
3422
3509
3031
2823
2412
1812
3226
2419
2824
2921
2829
2320
3214
2018
1925
nan
nan
nan
Numer osoby
2
2518
2206
2321
2016
3120
3505
2309
2214
2816
0520
2834
2323
2112
2326
2217
2116
3114
2121
1628
nan
nan
nan
nan
3
1232
2412
2220
2321
2029
3307
0636
2715
1722
3433
1919
2322
0921
0734
3920
2418
3123
1312
1710
nan
nan
nan
nan
4
2123
2011
2119
2019
2618
3408
1031
1723
2016
3228
2226
2421
3623
2333
2815
2323
2714
1415
2128
nan
nan
nan
nan
5
2430
2616
1819
2615
2919
2107
0227
1427
2809
2536
1821
2122
3319
0707
2919
2521
2705
1613
1322
nan
nan
nan
nan
6
1930
3330
1521
1125
3425
2007
1013
2718
1710
2715
2923
2024
2425
0815
2225
2223
2409
1118
2014
nan
nan
nan
nan
7
2625
1815
2019
2021
1931
3407
1038
1324
2120
3133
2122
1921
3717
0826
3917
2422
3122
1412
1828
nan
nan
nan
nan
8
2921
2118
1919
1728
3227
2907
2120
1619
2012
1123
3428
2118
1530
3331
1723
2021
3011
1424
2118
nan
nan
n

In [25]:
print(data_dict[rand_int])

         0        1     2     3     4     5     6     7     8     9   ...  \
0       NaN  Sesja 2   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
1   Nr O.B.        1     2     3     4     5     6     7     8     9  ...   
2         1     2023  1823  1527  1520  1516  1717  1816  2118  2115  ...   
3         2     2013  2101  2209  1707  2514  3227  1412  1819  2519  ...   
4         3     2520  2719  2619  2720  2417  2521  2721  2617  2517  ...   
5         4     3115  2517  2422  2320  2815  1312  2222  2028  2929  ...   
6         5     2718  2819  2128  2318  2519  3027  2228  3012  2516  ...   
7         6      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
8         7     3314  3310  2912  3114  0712  0512  3112  2312  2112  ...   
9         8     3735  2505  0538  1338  1138  1811  0938  2221  0417  ...   
10        9     3025  2613  1413  2723  1916  1316  1719  1322  2220  ...   
11       10     2507  2609  3010  2308  2806  3008  2813  2811  2506  ...   